
# **Hidden Markov Model (HMM) Part-of-Speech Tagger**
# **Implementation with Brute Force and Viterbi Decoding**



**1. Import Required Libraries**

In [1]:
import math

print("✓ Libraries loaded successfully")

✓ Libraries loaded successfully


**2. Training Data**

This training data is used to train the HMM model.
Each sentence is a list of (word, tag) tuples.

In [2]:
training_data = [
    [("The", "DET"), ("dog", "NOUN"), ("barks", "VERB")],
    [("The", "DET"), ("cat", "NOUN"), ("meows", "VERB")],
    [("A", "DET"), ("dog", "NOUN"), ("runs", "VERB")],
    [("The", "DET"), ("dog", "NOUN"), ("runs", "VERB")],
    [("A", "DET"), ("cat", "NOUN"), ("sleeps", "VERB")],
    [("The", "DET"), ("big", "ADJ"), ("dog", "NOUN"), ("barks", "VERB")],
    [("A", "DET"), ("small", "ADJ"), ("cat", "NOUN"), ("meows", "VERB")],
    [("The", "DET"), ("dog", "NOUN"), ("sleeps", "VERB")],
    [("The", "DET"), ("cat", "NOUN"), ("runs", "VERB")],
    [("A", "DET"), ("big", "ADJ"), ("dog", "NOUN"), ("sleeps", "VERB")],
    [("The", "DET"), ("small", "ADJ"), ("cat", "NOUN"), ("runs", "VERB")],
    [("Dogs", "NOUN"), ("bark", "VERB")],
    [("Cats", "NOUN"), ("meow", "VERB")],
    [("The", "DET"), ("dog", "NOUN"), ("in", "ADP"), ("the", "DET"), ("house", "NOUN"), ("barks", "VERB")],
    [("A", "DET"), ("cat", "NOUN"), ("on", "ADP"), ("the", "DET"), ("mat", "NOUN"), ("sleeps", "VERB")],
]

print(f"✓ Training data loaded: {len(training_data)} sentences")

✓ Training data loaded: 15 sentences


**3. Extract Unique POS Tags**

Extract all unique POS tags that appear in the training corpus.
These tags will serve as the hidden states in our HMM.

In [3]:
tags_set = set()
for sent in training_data:
    for word_tag_pair in sent:
        tags_set.add(word_tag_pair[1])

POS_list = sorted(list(tags_set))
print("POS Tags:", POS_list)

POS Tags: ['ADJ', 'ADP', 'DET', 'NOUN', 'VERB']


**4. Initial Probabilities**

This section computes the Initial Probabilities of each POS tag — the likelihood that a tag appears at the beginning of a sentence.

Since some tags may not occur at the start in the training data, Laplace (Add-One) smoothing is applied to avoid zero probabilities.

**Formula:**

$$P(\text{tag}) = \frac{\text{count}(\text{tag at start}) + 1}{\text{total sentences} + \text{num tags}}$$

**Explanation of terms:**

* **`count(tag at start)`** → number of sentences that start with this tag,

* **`total sentences`** → total number of sentences in the corpus,

* **`num tags`** → total number of unique POS tags.


The result is a probability distribution over all tags, representing how likely each tag is to appear as the first one in a sentence.

These probabilities define the initial state distribution of the Hidden Markov Model (HMM), serving as the starting point for predicting tag sequences.

In [4]:
def compute_initial_probabilities(data):
    start_tag_counts = {}

    for tag in POS_list:
        start_tag_counts[tag] = 1

    for sentence in data:
        first_tag = sentence[0][1]
        start_tag_counts[first_tag] += 1

    total = len(data) + len(POS_list)
    probabilities = {}

    for tag in POS_list:
        probabilities[tag] = start_tag_counts[tag] / total

    return probabilities

initial_probabilities = compute_initial_probabilities(training_data)
print("Initial Probabilities:", initial_probabilities)

Initial Probabilities: {'ADJ': 0.05, 'ADP': 0.05, 'DET': 0.7, 'NOUN': 0.15, 'VERB': 0.05}


**5. Count Tag Occurrences**

Count total occurrences of each tag in the training data.
Needed for normalizing transition and emission probabilities.

In [5]:
def count_tags(data):
    counts = {}
    for tag in POS_list:
        counts[tag] = 0

    for sentence in data:
        for _, tag in sentence:
            counts[tag] += 1

    return counts

tag_frequencies = count_tags(training_data)
print("Tag Frequencies:", tag_frequencies)

Tag Frequencies: {'ADJ': 4, 'ADP': 2, 'DET': 15, 'NOUN': 17, 'VERB': 15}


**6. Transition Probabilities**

This section computes the Transition Probabilities, which represent the likelihood of moving from one POS tag to another in a sentence.

In the context of an HMM, these probabilities model the dependency between consecutive tags — for example, how likely it is that a NOUN is followed by a VERB.

Since some tag transitions may never appear in the training data, Laplace (Add-One) smoothing is applied to ensure that every possible transition receives a non-zero probability.

**Formula:** $$P(\text{tag}_i \mid \text{tag}_{i-1}) = \frac{\text{count}(\text{tag}_{i-1}, \text{tag}_i) + 1}{\text{count}(\text{tag}_{i-1}) + \text{num tags}}$$

**Explanation of terms:**

* **`count(tag_{i-1}, tag_i)`** → number of times the pair of consecutive tags occurs in the data,

* **`count(tag_{i-1})`** → number of occurrences of the previous tag,

* **`num tags`** → total number of unique POS tags.


The resulting transition matrix provides a full mapping of how likely each tag is to follow another.

These probabilities form a core component of the HMM structure, as they describe the relationships between hidden states and are later used by the Viterbi algorithm to predict the most probable tag sequence.

In [6]:
def compute_transition_probabilities(data, tag_counts):
    transitions = {}

    for tag1 in POS_list:
        transitions[tag1] = {}
        for tag2 in POS_list:
            transitions[tag1][tag2] = 1

    for sentence in data:
        for idx in range(len(sentence) - 1):
            prev_tag = sentence[idx][1]
            next_tag = sentence[idx + 1][1]
            transitions[prev_tag][next_tag] += 1

    for prev_tag in POS_list:
        denominator = tag_counts[prev_tag] + len(POS_list)
        for curr_tag in POS_list:
            transitions[prev_tag][curr_tag] /= denominator

    return transitions

transition_probabilities = compute_transition_probabilities(training_data, tag_frequencies)

print("Transition Probabilities:")
for tag1, trans_dict in transition_probabilities.items():
    formatted_trans = {t2: f"{p:.2f}" for t2, p in trans_dict.items()}
    print(f"  '{tag1}': {formatted_trans}")

Transition Probabilities:
  'ADJ': {'ADJ': '0.11', 'ADP': '0.11', 'DET': '0.11', 'NOUN': '0.56', 'VERB': '0.11'}
  'ADP': {'ADJ': '0.14', 'ADP': '0.14', 'DET': '0.43', 'NOUN': '0.14', 'VERB': '0.14'}
  'DET': {'ADJ': '0.25', 'ADP': '0.05', 'DET': '0.05', 'NOUN': '0.60', 'VERB': '0.05'}
  'NOUN': {'ADJ': '0.05', 'ADP': '0.14', 'DET': '0.05', 'NOUN': '0.05', 'VERB': '0.73'}
  'VERB': {'ADJ': '0.05', 'ADP': '0.05', 'DET': '0.05', 'NOUN': '0.05', 'VERB': '0.05'}


**7․ Extract Vocabulary**

Extract all unique words from the training data to build vocabulary.

In [7]:
def extract_vocabulary(data):
    words = []
    for sentence in data:
        for word, _ in sentence:
            if word not in words:
                words.append(word)
    return words

vocabulary = extract_vocabulary(training_data)
print("Vocabulary:", vocabulary)

Vocabulary: ['The', 'dog', 'barks', 'cat', 'meows', 'A', 'runs', 'sleeps', 'big', 'small', 'Dogs', 'bark', 'Cats', 'meow', 'in', 'the', 'house', 'on', 'mat']


**8. Emission Probabilities**



This section computes the Emission Probabilities, which represent the likelihood of a word being generated (emitted) from a specific POS tag.

In an HMM-based POS tagger, emission probabilities define the relationship between hidden states (tags) and observed words.
They tell the model how likely it is to see a word given that the current hidden state (tag) is known.

Because some word-tag combinations might never appear in the training data, Laplace (Add-One) smoothing is applied to avoid zero probabilities.

**Formula:** $$P(\text{word} \mid \text{tag}) = \frac{\text{count}(\text{word}, \text{tag}) + 1}{\text{count}(\text{tag}) + \text{num words}}$$

**Explanation of terms:**

* **`count(word, tag)`** → number of times a specific word appears with a given tag,

* **`count(tag)`** → total occurrences of the tag in the corpus,

* **`num words`** → total number of unique words in the vocabulary.

These probabilities form the observation model of the HMM, allowing it to estimate which words are most characteristic of each tag when predicting unseen sentences.

In [8]:
def compute_emission_probabilities(data, tag_counts, vocab):
    emissions = {}

    for tag in POS_list:
        emissions[tag] = {}
        for word in vocab:
            emissions[tag][word] = 1

    for sentence in data:
        for word, tag in sentence:
            emissions[tag][word] += 1

    vocab_size = len(vocab)
    for tag in POS_list:
        denominator = tag_counts[tag] + vocab_size
        for word in vocab:
            emissions[tag][word] /= denominator

    return emissions

emission_probabilities = compute_emission_probabilities(training_data, tag_frequencies, vocabulary)

print("Emission Probabilities:")
for tag, emission_dict in emission_probabilities.items():
    formatted_probs = {word: f"{prob:.2f}" for word, prob in emission_dict.items()}
    print(f"  '{tag}': {formatted_probs}")

Emission Probabilities:
  'ADJ': {'The': '0.04', 'dog': '0.04', 'barks': '0.04', 'cat': '0.04', 'meows': '0.04', 'A': '0.04', 'runs': '0.04', 'sleeps': '0.04', 'big': '0.13', 'small': '0.13', 'Dogs': '0.04', 'bark': '0.04', 'Cats': '0.04', 'meow': '0.04', 'in': '0.04', 'the': '0.04', 'house': '0.04', 'on': '0.04', 'mat': '0.04'}
  'ADP': {'The': '0.05', 'dog': '0.05', 'barks': '0.05', 'cat': '0.05', 'meows': '0.05', 'A': '0.05', 'runs': '0.05', 'sleeps': '0.05', 'big': '0.05', 'small': '0.05', 'Dogs': '0.05', 'bark': '0.05', 'Cats': '0.05', 'meow': '0.05', 'in': '0.10', 'the': '0.05', 'house': '0.05', 'on': '0.10', 'mat': '0.05'}
  'DET': {'The': '0.26', 'dog': '0.03', 'barks': '0.03', 'cat': '0.03', 'meows': '0.03', 'A': '0.18', 'runs': '0.03', 'sleeps': '0.03', 'big': '0.03', 'small': '0.03', 'Dogs': '0.03', 'bark': '0.03', 'Cats': '0.03', 'meow': '0.03', 'in': '0.03', 'the': '0.09', 'house': '0.03', 'on': '0.03', 'mat': '0.03'}
  'NOUN': {'The': '0.03', 'dog': '0.22', 'barks': '0.03

**9. HMM POS Tagger Class**

**HMMPOSTagger (partial) – Key Points**

* **`__init__`**: Initializes empty structures for tags, vocabulary, counts, and probabilities.
* **`train(data)`**: Fills `tag_set`, `word_set`, counts, and computes initial (`pi`), transition (`A`), and emission (`B`) probabilities using Add-One smoothing.
* **`recursive_generate_sequences(seq, pos, sent_len, result)`**: Recursively generates all possible tag sequences of length `sent_len` (for brute-force decoding).
* **`brute_force_decode(sentence)`**: Finds the most probable tag sequence by checking all possible sequences:


$$ P(\text{tags, words}) = P(\text{tag}_1) \cdot P(\text{word}_1 \mid \text{tag}*1) \cdot \prod*{i=2}^{n} P(\text{tag}*i \mid \text{tag}*{i-1}) \cdot P(\text{word}_i \mid \text{tag}_i) $$


* Uses a small default probability $( 1 \times 10^{-6} )$  for unknown words.


In [9]:
import math

class HMMPOSTagger:
    def __init__(self):
        self.tag_set = []
        self.word_set = []
        self.pi = {}
        self.A = {}
        self.B = {}
        self.tag_totals = {}

    def train(self, data):
        all_tags = set()
        for sentence in data:
            for _, tag in sentence:
                all_tags.add(tag)
        self.tag_set = sorted(list(all_tags))

        self.word_set = extract_vocabulary(data)
        self.tag_totals = count_tags(data)
        self.pi = compute_initial_probabilities(data)
        self.A = compute_transition_probabilities(data, self.tag_totals)
        self.B = compute_emission_probabilities(data, self.tag_totals, self.word_set)

    def recursive_generate_sequences(self, seq, pos, sent_len, result):
        if pos == sent_len:
            result.append(seq[:])
            return

        for t in self.tag_set:
            seq.append(t)
            self.recursive_generate_sequences(seq, pos + 1, sent_len, result)
            seq.pop()

    def brute_force_decode(self, sentence):
        all_tag_sequences = []
        self.recursive_generate_sequences([], 0, len(sentence), all_tag_sequences)

        max_probability = 0
        optimal_sequence = None

        for tag_seq in all_tag_sequences:
            prob = self.pi[tag_seq[0]]
            prob *= self.B[tag_seq[0]].get(sentence[0], 1e-6)

            for pos in range(1, len(sentence)):
                prob *= self.A[tag_seq[pos - 1]][tag_seq[pos]]
                prob *= self.B[tag_seq[pos]].get(sentence[pos], 1e-6)

            if prob > max_probability:
                max_probability = prob
                optimal_sequence = tag_seq

        return optimal_sequence

**10. Viterbi Decoding Function**

The `viterbi_decode` function finds the most probable POS tag sequence for a given sentence using the **Viterbi algorithm**, an efficient dynamic programming approach.

**Key Steps:**

1. **Initialization:**
   For the first word, compute log-probabilities combining the initial tag probabilities (`pi`) and emission probabilities (`B`).

2. **Recursion:**
   For each subsequent word, compute the best score for each tag by considering all possible previous tags. Store both the score (`viterbi`) and the best predecessor tag (`path`) for backtracking.

   $$\text{score(tag)} = \max_{\text{prev\_tag}} \Big( \text{viterbi}[pos-1][\text{prev\_tag}] + \log P(\text{tag} \mid \text{prev\_tag}) + \log P(\text{word} \mid \text{tag})\Big)$$

3. **Termination / Backtracking:**
   * Start from the tag with the highest score at the last word.
   * Backtrack using `path` to reconstruct the optimal tag sequence.

**Advantages:**
* Guarantees finding the globally optimal sequence.
* Much more efficient than brute-force decoding: $O(n \cdot t^2)$ instead of $O(t^n)$.

**Unknown Words Handling:**
* Words not seen during training are assigned a small probability (`1e-6`) to avoid zero probabilities.

In [10]:
def viterbi_decode(sentence, tag_set, pi, A, B):
    n = len(sentence)
    viterbi = []
    path = []

    # Initialization
    initial_scores = {}
    initial_pointers = {}
    for tag in tag_set:
        emission_val = B[tag].get(sentence[0], 1e-6)
        initial_scores[tag] = math.log(pi[tag]) + math.log(emission_val)
        initial_pointers[tag] = None

    viterbi.append(initial_scores)
    path.append(initial_pointers)

    # Recursion
    for pos in range(1, n):
        current_scores = {}
        current_pointers = {}
        word = sentence[pos]

        for tag in tag_set:
            best_score = float("-inf")
            best_predecessor = None

            for prev_tag in tag_set:
                trans_prob = A[prev_tag][tag]
                emit_prob = B[tag].get(word, 1e-6)
                score = viterbi[pos - 1][prev_tag] + math.log(trans_prob) + math.log(emit_prob)

                if score > best_score:
                    best_score = score
                    best_predecessor = prev_tag

            current_scores[tag] = best_score
            current_pointers[tag] = best_predecessor

        viterbi.append(current_scores)
        path.append(current_pointers)

    # Termination: backtrack to find the best sequence
    final_tag = max(viterbi[-1], key=viterbi[-1].get)
    result = [final_tag]
    for pos in range(n - 1, 0, -1):
        result.append(path[pos][result[-1]])

    result.reverse()
    return result

**11. Train the Model**

Initialize the tagger and train it on the training corpus.

In [11]:
hmm_tagger = HMMPOSTagger()
hmm_tagger.train(training_data)
print("Model training complete!")
print(f"Tags in model: {len(hmm_tagger.tag_set)}")
print(f"Vocabulary size: {len(hmm_tagger.word_set)}")

Model training complete!
Tags in model: 5
Vocabulary size: 19


**12. Test the Tagger**

Evaluate the tagger on test sentences using both algorithms.
Both methods should produce identical results.

In [13]:
test_sentences = [
    ["Small", "cat", "sleeps"],
    ["The", "young", "girl", "dances", "gracefully"],
    ["The", "moon", "glows"],
    ["A", "tiny", "rabbit", "nibbles", "carrots"],
    ["The", "cheerful", "child", "plays", "outside"],
    ["Dogs", "run", "fast"],
    ["The", "boy", "writes", "a", "letter"],
    ["A", "colorful", "butterfly", "flies", "away"]
]

for sent in test_sentences:
    brute_tags = hmm_tagger.brute_force_decode(sent)
    viterbi_tags = viterbi_decode(sent, hmm_tagger.tag_set, hmm_tagger.pi,
                                  hmm_tagger.A, hmm_tagger.B)

    print(f"Sentence: {' '.join(sent)}")
    print(f"  Brute Force: {', '.join(brute_tags)}")
    print(f"  Viterbi:     {', '.join(viterbi_tags)}\n")

Sentence: Small cat sleeps
  Brute Force: DET, NOUN, VERB
  Viterbi:     DET, NOUN, VERB

Sentence: The young girl dances gracefully
  Brute Force: DET, NOUN, ADP, DET, NOUN
  Viterbi:     DET, NOUN, ADP, DET, NOUN

Sentence: The moon glows
  Brute Force: DET, NOUN, VERB
  Viterbi:     DET, NOUN, VERB

Sentence: A tiny rabbit nibbles carrots
  Brute Force: DET, NOUN, ADP, DET, NOUN
  Viterbi:     DET, NOUN, ADP, DET, NOUN

Sentence: The cheerful child plays outside
  Brute Force: DET, NOUN, ADP, DET, NOUN
  Viterbi:     DET, NOUN, ADP, DET, NOUN

Sentence: Dogs run fast
  Brute Force: DET, NOUN, VERB
  Viterbi:     DET, NOUN, VERB

Sentence: The boy writes a letter
  Brute Force: DET, NOUN, ADP, DET, NOUN
  Viterbi:     DET, NOUN, ADP, DET, NOUN

Sentence: A colorful butterfly flies away
  Brute Force: DET, NOUN, ADP, DET, NOUN
  Viterbi:     DET, NOUN, ADP, DET, NOUN



**13. Complexity Analysis (Bonus)**

**TIME COMPLEXITY COMPARISON**

**1. BRUTE FORCE method:**
   * Generates all possible tag sequences
   * If sentence has n words and t tags exist:
     Time complexity: O(t^n × n)

     * t^n sequences to generate
     * n operations for each sequence

   Example: 5 tags, 10 words → 5^10 = 9,765,625 sequences

   Problem: Exponential growth - impractical for long sentences

**2. VITERBI algorithm (Dynamic Programming):**

   * Uses dynamic programming to store solutions to subproblems
   * Time complexity: O(n × t^2)

     * n positions in sentence
     * t^2 transitions for each position

   Example: 5 tags, 10 words → 10 × 5^2 = 250 operations

   Advantage: Multiple orders of magnitude faster for long sentences

**SUMMARY:**

* Brute Force: O(t^n × n) - Exponential, but easy to understand
* Viterbi: O(n × t^2) - Polynomial, efficient
* Viterbi is always used in real POS tagging systems


In [16]:
print("\n" + "="*60)
print("COMPLEXITY ANALYSIS")
print("="*60)

# Parameters
n_words = 10
n_tags = len(hmm_tagger.tag_set)

# Compute operations
brute_force_ops = n_tags ** n_words * n_words
viterbi_ops = n_words * n_tags ** 2

# Print parameters
print(f"\nParameters:")
print(f"  - Number of words (n): {n_words}")
print(f"  - Number of tags (t): {n_tags}")

# Brute Force Complexity
print(f"\nBrute Force - O(t^n × n):")
print(f"  Operations: {brute_force_ops:,}")

# Viterbi Complexity
print(f"\nViterbi - O(n × t^2):")
print(f"  Operations: {viterbi_ops:,}")

# Speedup ratio
speedup = brute_force_ops / viterbi_ops
print(f"\nRatio: Viterbi is {speedup:,.0f}x faster")

print("\n" + "="*60)
print("PROGRAM COMPLETED")
print("="*60)


COMPLEXITY ANALYSIS

Parameters:
  - Number of words (n): 10
  - Number of tags (t): 5

Brute Force - O(t^n × n):
  Operations: 97,656,250

Viterbi - O(n × t^2):
  Operations: 250

Ratio: Viterbi is 390,625x faster

PROGRAM COMPLETED
